### Load Your Documents

In [1]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    "https://blog.langchain.dev/langchain-v0-1-0/"
)

documents = loader.load()

### Instantiate Embedding Model

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

### Instantiate LLM

In [3]:
from langchain_openai import OpenAI
import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai_llm = OpenAI(temperature=0)

### Document Splitter

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1250,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex = False
)
#
split_docs = text_splitter.split_documents(documents)
print(len(split_docs))

23


### Instantiate the Vectorstore

In [5]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma(embedding_function=embeddings,
                     persist_directory="Vectorstore/chromadb",
                     collection_name="full_documents")

### Load and persist the split documents into the vectorstore

In [6]:
vectorstore.add_documents(split_docs)
vectorstore.persist()

### Instantiate the Keyword / Sparse embeddings model

In [7]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
#
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k=10

### Instantiate Reranker — Cross Encoders

In [20]:
from __future__ import annotations
from typing import Dict, Optional, Sequence
from langchain.schema import Document
from langchain.pydantic_v1 import Extra, root_validator

from langchain.callbacks.manager import Callbacks
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor

from sentence_transformers import CrossEncoder
# from config import bge_reranker_large

class BgeRerank(BaseDocumentCompressor):
    model_name:str = 'BAAI/bge-reranker-large'
    """Model name to use for reranking."""
    top_n: int = 3
    """Number of documents to return."""
    model:CrossEncoder = CrossEncoder(model_name)
    """CrossEncoder instance to use for reranking."""

    def bge_rerank(self,query,docs):
        model_inputs =  [[query, doc] for doc in docs]
        scores = self.model.predict(model_inputs)
        results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        return results[:self.top_n]


    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        """
        Compress documents using BAAI/bge-reranker models.

        Args:
            documents: A sequence of documents to compress.
            query: The query to use for compressing the documents.
            callbacks: Callbacks to run during the compression process.

        Returns:
            A sequence of compressed documents.
        """
        if len(documents) == 0:  # to avoid empty api call
            return []
        doc_list = list(documents)
        _docs = [d.page_content for d in doc_list]
        results = self.bge_rerank(query, _docs)
        final_results = []
        for r in results:
            doc = doc_list[r[0]]
            doc.metadata["relevance_score"] = r[1]
            final_results.append(doc)
        return final_results

model.safetensors:  13%|█▎        | 294M/2.24G [35:19<25:08, 1.29MB/s]  Error while downloading from https://cdn-lfs.huggingface.co/repos/66/eb/66eb55346eb5bfcafa6de366b299d05cbc1e2fd5b7453b104a6c20eb78660624/c5ae4e262c60ed2fb507ec587358285377ec36ee0a2e6da029f0534272b06d36?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1709038489&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTAzODQ4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Ni9lYi82NmViNTUzNDZlYjViZmNhZmE2ZGUzNjZiMjk5ZDA1Y2JjMWUyZmQ1Yjc0NTNiMTA0YTZjMjBlYjc4NjYwNjI0L2M1YWU0ZTI2MmM2MGVkMmZiNTA3ZWM1ODczNTgyODUzNzdlYzM2ZWUwYTJlNmRhMDI5ZjA1MzQyNzJiMDZkMzY%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=JH%7E-lMchQY1aIYwmt1xy5RKxmeLcV%7EyYDDRpf6RgoHB9-YGnK2A7IrEHykiOIzGhx-GguWRnOSaJZ8-rgMtEz4F6DE6erNni-UOI2sLiq5Avtr-2syLE69XgAGcFmfY3TKqiVq4GMqPfOnd-HgcAxUGyAiZHF7o5SL3DKE

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Max retries exceeded with url: /repos/66/eb/66eb55346eb5bfcafa6de366b299d05cbc1e2fd5b7453b104a6c20eb78660624/c5ae4e262c60ed2fb507ec587358285377ec36ee0a2e6da029f0534272b06d36?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1709038489&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTAzODQ4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Ni9lYi82NmViNTUzNDZlYjViZmNhZmE2ZGUzNjZiMjk5ZDA1Y2JjMWUyZmQ1Yjc0NTNiMTA0YTZjMjBlYjc4NjYwNjI0L2M1YWU0ZTI2MmM2MGVkMmZiNTA3ZWM1ODczNTgyODUzNzdlYzM2ZWUwYTJlNmRhMDI5ZjA1MzQyNzJiMDZkMzY~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=JH~-lMchQY1aIYwmt1xy5RKxmeLcV~yYDDRpf6RgoHB9-YGnK2A7IrEHykiOIzGhx-GguWRnOSaJZ8-rgMtEz4F6DE6erNni-UOI2sLiq5Avtr-2syLE69XgAGcFmfY3TKqiVq4GMqPfOnd-HgcAxUGyAiZHF7o5SL3DKEOEy9RAkSRDe

### Instantiate a Contextual Compression Pipeline

In [ ]:
from langchain_community.document_transformers.embeddings_redundant_filter import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_transformers.long_context_reorder import LongContextReorder
from langchain.retrievers.multi_query import MultiQueryRetriever
#
vs_retriever = vectorstore.as_retriever(search_kwargs={"k":10})
#

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever,vs_retriever],
                                       weight=[0.5,0.5])
#

redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
#
reordering = LongContextReorder()
#
reranker = BgeRerank()
#
pipeline_compressor = DocumentCompressorPipeline(transformers=[redundant_filter,reordering,reranker])
#
compression_pipeline = ContextualCompressionRetriever(base_compressor=pipeline_compressor,
                                                      base_retriever=ensemble_retriever)

### Helper function to display retrieved documents

In [ ]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n + {d.page_content}" for i,d in enumerate(docs)])
  )

In [ ]:
pretty_print_docs(vs_retriever.get_relevant_documents("What are the major changes in v 0.1.0?"))

In [ ]:
docs = compression_pipeline.get_relevant_documents("What are the major changes in v 0.1.0?")
pretty_print_docs(docs)

### Define an Advanced RAG

In [ ]:
from langchain.chains import RetrievalQA
#
qa_advanced = RetrievalQA.from_chain_type(llm=openai_llm,
                                 chain_type="stuff",
                                 retriever=compression_pipeline,
                                 return_source_documents=True)
#
qa_adv_response = qa_advanced("What are the major changes in v 0.1.0?")  
qa_adv_response["result"]